### Without LLMs-based Evolution on Jazz

In [ ]:
import networkx as nx
import numpy as np
from scipy.sparse.linalg import eigsh

def score_nodes(adj_matrix):
    G = nx.from_numpy_array(adj_matrix)
    scored_nodes = {node: 0 for node in G.nodes()}
    centralities = nx.betweenness_centrality(G)
    for node, centrality in centralities.items():
        scored_nodes[node] += centrality
    LCC = G.subgraph(max(nx.connected_components(G), key=len))
    L = nx.laplacian_matrix(LCC)
    eigenvalues, eigenvectors = eigsh(L.astype(np.float32), k=2, which='SM')
    Fiedler = eigenvectors[:, 1]
    H = nx.Graph()
    for u, v in LCC.edges():
        if Fiedler[list(LCC.nodes()).index(u)] * Fiedler[list(LCC.nodes()).index(v)] <= 0.0:
            H.add_edge(u, v)
    for v in H.nodes():
        scored_nodes[v] += 1.0 / H.degree(v)
    cover = nx.algorithms.approximation.min_weighted_vertex_cover(H)
    for v in cover:
        scored_nodes[v] += 1
    return scored_nodes

### Without Manual Initialization on Jazz

In [ ]:
import networkx as nx

def score_nodes(adj_matrix):
    num_nodes = adj_matrix.shape[0]
    adjacency_matrix = adj_matrix.copy()
    scored_nodes = {}

    G = nx.Graph(adjacency_matrix)
    betweenness = nx.betweenness_centrality(G)

    for node in range(num_nodes):
        neighbors = np.where(adjacency_matrix[node] == 1)[0]
        num_neighbors = len(neighbors)

        if num_neighbors < 2:
            score = 0.0
        else:
            sum_scores = 0.0
            for i in range(num_neighbors):
                for j in range(i + 1, num_neighbors):
                    if adjacency_matrix[neighbors[i], neighbors[j]] == 1:
                        sum_scores += betweenness[node] * (1.0 / (num_neighbors * (num_neighbors - 1)))

            score = sum_scores / num_neighbors

        scored_nodes[node] = score

    return scored_nodes

### Without Population Management on Jazz

In [ ]:
import numpy as np
import networkx as nx
from scipy.sparse.linalg import eigsh

def score_nodes(edge_matrix):
    adjacency_matrix = np.array(edge_matrix)
    num_nodes = adjacency_matrix.shape[0]
    node_degree_map = np.sum(adjacency_matrix, axis=0)
    clustering_coefficients = np.zeros(num_nodes)
    for node in range(num_nodes):
        neighbors = np.where(adjacency_matrix[node] == 1)[0]
        num_neighbors = len(neighbors)
        if num_neighbors < 2:
            clustering_coefficients[node] = 0.0
        else:
            num_connected_pairs = 0
            for i in range(num_neighbors):
                for j in range(i + 1, num_neighbors):
                    if adjacency_matrix[neighbors[i], neighbors[j]] == 1:
                        num_connected_pairs += 1
            clustering_coefficients[node] = (2.0 * num_connected_pairs) / (num_neighbors * (num_neighbors - 1))
    
    G = nx.from_numpy_array(adjacency_matrix)
    LCC = G.subgraph(max(nx.connected_components(G), key=len))
    
    ii = {v: i for i, v in enumerate(LCC.nodes())}
    L = nx.normalized_laplacian_matrix(LCC)
    eigenvalues, eigenvectors = eigsh(L.astype(np.float32), k=2, which='SM', maxiter=1000 * L.shape[0])
    Fiedler = eigenvectors[:, 1]

    H = nx.Graph([(u, v) for u, v in LCC.edges() if Fiedler[ii[u]] * Fiedler[ii[v]] <= 0.0])
    for v in H.nodes():
        H.nodes[v]['weight'] = 1.0 / H.degree(v)
    cover = list(nx.algorithms.approximation.min_weighted_vertex_cover(H, weight='weight'))
    max_degree = max([G.degree(v) for v in G.nodes() if v not in cover])
    min_weight = min(H.nodes[v]['weight'] for v in H.nodes())
    scored_nodes = {v: H.nodes[v]['weight'] if v in cover else clustering_coefficients[node_id] * min_weight for v, node_id in enumerate(G.nodes())}
    
    return scored_nodes